In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
def parse_champs(game):
    import pickle
    import numpy as np
    with open('data/champ_dict.pkl', 'rb') as f:
        champ_dict = pickle.load(f)
        
    picked = []
    picked.append([champ_dict[champ] for champ in game])
    picked = picked[0]
    
    five_hot1 = np.zeros((len(champ_dict),), dtype=int)
    for j in picked[0:5]:
        five_hot1[j] = 1
    five_hot2 = np.zeros((len(champ_dict),), dtype=int)
    for k in picked[5:10]:
        five_hot2[k] = 1
    return np.concatenate((five_hot1, five_hot2))

In [3]:
features = [
    nn.Linear(270, 128),
    nn.ReLU(),
    nn.Dropout(p=0.2),
    nn.Linear(128, 128),
    nn.ReLU(),
    nn.Dropout(p=0.2),
    nn.Linear(128, 2)]

def load_model(path):
    model = nn.Sequential(*features).cuda()
    model.load_state_dict(torch.load(f'models/{path}'))
    model.eval()
    return model

In [4]:
net = load_model('sixty.pth')

In [5]:
# returns [blue win probability, red win probability]

def custom_test(net_, game):
    ten_hot = parse_champs(game)
    custom_X = torch.tensor(ten_hot, dtype=torch.float)
    
    net_.eval()
    with torch.no_grad():
        output = net_(custom_X.cuda())
    net_.train()
    
    return F.softmax(output)

In [6]:
custom_test(net, ['Renekton', 'Lee Sin', 'Syndra', 'Draven', 'Blitzcrank', 'Ornn', 'Nidalee', 'Orianna', 'Ashe', 'Sett']) # expected red win
# sinner comp loses :)

<ipython-input-5-84e7400dbd4b>:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(output)


tensor([0.2621, 0.7379], device='cuda:0')

In [7]:
custom_test(net, ['Jayce',"Kha'Zix",'LeBlanc','Caitlyn','Brand','Ornn','Sett','Cassiopeia','Senna','Pantheon']) # expected red win

<ipython-input-5-84e7400dbd4b>:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(output)


tensor([0.4471, 0.5529], device='cuda:0')